In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd 

In [2]:
def data_rbref_df(a,year,b,category,c,number_of_headers):
    url=(a+year+b+category+c)
    print(url)
    page = urllib.request.urlopen(url)
    soup = bs(page)
    #print(soup)
    td_s = soup.body.findAll('td') # targets the data table at the url page 
    data_cells = (str(td_s)).split(',') # gives me each data cell as an index in a list 
    #print(data_cells[11])
    #print(len(data_cells))
    table_header_temp=[]
    table_data_temp=[]
    table_header=[]
    table_data=[]

    #find in the html text data headers and data itself 
    for cells in data_cells:
        table_header_temp.append(re.findall('data-stat="\w+"', cells))
        table_data_temp.append(re.findall('>(.*?)<', cells))

    table_header = [(str(item).split('=')[1][1:-3]) for item in table_header_temp]

    for i in table_data_temp:
        table_data.append(''.join(s for s in i if s)) # omit empty items in list


    #print(type(table_header[17]))
    #print(type(table_data[17]))

    keys_header = table_header[0:number_of_headers]
    values_data = [[] for i in range(len(keys_header))]
    number_of_games=38
    #print(number_of_games)

    for i in range(len(keys_header)):
        for j in range(number_of_games):
            values_data[i].append(table_data[i+j*len(keys_header)])

    my_dict = dict(zip(keys_header, values_data))
    df = pd.DataFrame.from_dict(my_dict)
    df = df.drop(columns=['match_report'])
    return df

In [16]:
#'https://fbref.com/en/squads/206d90db/2020-2021/matchlogs/c12/keeper/Barcelona-Scores-and-Fixtures-La-Liga'
cat=['keeper', 'defense', 'possession', 'passing', 'passing_types', 'gca', 'shooting', 'misc' ]
num_headers_percat=[35,32,34,30,34,23,24,25] #17 for schedule
years=['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

a='https://fbref.com/en/squads/206d90db/'
b='/matchlogs/c12/'
category='schedule'
c='/Barcelona-Scores-and-Fixtures-La-Liga'
list_dfs=[]

for i in range(len(cat)):
    list_dfs.append(data_rbref_df(a,'2017-2018',b,cat[i],c,num_headers_percat[i]))

temp_df = pd.concat(list_dfs[1:], axis=1)
temp_df = temp_df.drop(columns=['start_time', 'round', 'dayofweek', 'venue', 'result', 'goals_for','goals_against', 'opponent'])
df = pd.concat([list_dfs[0],temp_df],axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)


    


https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/keeper/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/defense/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/possession/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/passing/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/passing_types/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/gca/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/shooting/Barcelona-Scores-and-Fixtures-La-Liga
https://fbref.com/en/squads/206d90db/2017-2018/matchlogs/c12/misc/Barcelona-Scores-and-Fixtures-La-Liga
   start_time         round dayofweek venue result goals_for goals_against  \
0      20:15    Matchweek 1       Sun  Home      W         2    